In [1]:
%matplotlib qt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from copy import copy

from gauss import Gaussian
from Estimator_Kalman import Kalman
from Estimator_BayesFilter import Bayes
from pgm import pgm_estimator


dt = 1
xt = np.array(np.zeros((2,11)))
zt = np.array(np.zeros((2,11)))
    #ut = np.array([0,0])
A = np.array([[1,0],[0,1]])
B = np.array([[dt, 0],[0, dt]])
C = np.array([[1,0],[0,1]])
    
R = np.array([[0.1, 0],[0, 0.1]])
Q = np.array([[0.1 , 0],[0, 0.1]])
    
noiseMean = np.array([0,0])
    
u = np.array(np.zeros((2,10)))
xt[:,0] = np.round(np.random.multivariate_normal(noiseMean, R), decimals = 4)

beliefs = []
meanX0 = np.array([0,0])
covX0 = np.array([[0.1,0],[0,0.1]])
beliefX0 = Gaussian(['xt1','xt2'], meanX0, covX0)

for i in range(0, len(u[0])):        
    u[0, i] = np.random.normal(2, 1)
    u[1, i] = np.random.normal(3,1)
    u = np.round(u)
    xt[:,i+1] = np.round(A.dot(xt[:,i]) + B.dot(u[:,i]) + np.random.multivariate_normal(noiseMean, R), decimals = 4)
    zt[:, i+1] = np.round(xt[:,i+1] + C.dot(np.random.multivariate_normal(noiseMean, Q)), decimals = 4)    

In [2]:
##using the kalman filter##
myKalman = Kalman(A, B, C, R, Q)
    
estimatedMeans = np.array(np.zeros((2,11)))
estimatedSigs = np.array(np.zeros((2,2,11)))

estimatedSigs[:,: , 0] = np.array([[0.1, 0],[0, 0.1]])
    
estimatedLambdas = np.array(np.zeros((2,11)))
estimatedVs = np.array(np.zeros((2,2,11)))

fig0 = plt.figure(0)
ax0 = plt.subplot(111, aspect = 'equal')

for i in range (0, len(u[0])):
        #print(i)
    estimatedMeans[:, i+1], estimatedSigs[:,:,i+1] = myKalman.kalman_filter(estimatedMeans[:, i], estimatedSigs[:,:,i] , u[:, i], zt[:, i+1])
        
for i in range (0, 11):
    estimatedLambdas[:, i], estimatedVs[:,:, i] = np.linalg.eig(estimatedSigs[:,:,i])
    estimatedLambdas[:, i] = np.sqrt(estimatedLambdas[:, i])
        
    for j in range(3, 4):
        ell = Ellipse(xy = (estimatedMeans[0, i],estimatedMeans[1, i]), 
                        width = estimatedLambdas[0 , i]*j*2, 
                        height = estimatedLambdas[1, i]*j*2, 
                        angle = np.rad2deg(np.arccos(estimatedVs[0,0,i])), color = 'black')
            
        ell.set_facecolor('none')
        ax0.add_artist(ell)
#print(estimatedVs)
#print(estimatedMeans)
#print(estimatedSigs[:,:,10])
    #plt.scatter(estimatedMeans[0], estimatedMeans[1], c = "blue", marker = "o") 

plt.scatter(xt[0], xt[1], c = "red", marker = "x")
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()

In [3]:
##Using the bayes filter##

beliefs.append(beliefX0)

bayesFilter = Bayes(A, B, C, R, Q)
    
estimatedLambdas = np.array(np.zeros((2,11)))
estimatedVs = np.array(np.zeros((2,2,11)))

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111, aspect = 'equal')

for i in range (0, len(u[0])):
    myDist = bayesFilter.bayes_filter(beliefXtPrev= beliefs[i], control_t= u[:, i], z_t = zt[:, i+1])    
    beliefs.append(myDist)


for i in range (0, 11):    
    beliefs[i].updateCov()
   # print(beliefs[i].cov)   
    estimatedLambdas[:, i], estimatedVs[:,:, i] = np.linalg.eig(beliefs[i].cov)
    estimatedLambdas[:, i] = np.sqrt(estimatedLambdas[:, i])
        
   
    for j in range(3, 4):
        ell = Ellipse(xy = (beliefs[i].mean[0],beliefs[i].mean[1]), 
                        width = estimatedLambdas[0 , i]*j*2, 
                        height = estimatedLambdas[1, i]*j*2, 
                        angle = np.rad2deg(np.arccos(estimatedVs[0,0,i])), color = 'black')
            
        ell.set_facecolor('none')
        ax1.add_artist(ell)

plt.scatter(xt[0], xt[1], c = "red", marker = "x")
#plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()


In [4]:
## PGM approach ##

myPGM = pgm_estimator(A, B, C, R, Q, zt, u, beliefX0)

estimatedLambdas = np.array(np.zeros((2,11)))
estimatedVs = np.array(np.zeros((2,2,11)))

fig2 = plt.figure(2)
ax2 = fig2.add_subplot(111, aspect = 'equal')

for i in range (0, 11):    
    #print(i)
    #print(myPGM.beliefs[i].cov)   
    estimatedLambdas[:, i], estimatedVs[:,:, i] = np.linalg.eig(myPGM.beliefs[i].cov)
    estimatedLambdas[:, i] = np.sqrt(estimatedLambdas[:, i])
        
   
    for j in range(3, 4):
        ell = Ellipse(xy = (myPGM.beliefs[i].mean[0], myPGM.beliefs[i].mean[1]), 
                        width = estimatedLambdas[0 , i]*j*2, 
                        height = estimatedLambdas[1, i]*j*2, 
                        angle = np.rad2deg(np.arccos(estimatedVs[0,0,i])), color = 'black')
            
        ell.set_facecolor('none')
        ax2.add_artist(ell)

plt.scatter(xt[0], xt[1], c = "red", marker = "x")
#plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()

